<a href="https://colab.research.google.com/github/aliochilov/Aviakompaniya/blob/main/Aviakompaniya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [84]:

test_set = pd.read_csv('https://github.com/aliochilov/Aviakompaniya/blob/main/test_dataset.csv?raw=True')
train_set = pd.read_csv('https://github.com/aliochilov/Aviakompaniya/blob/main/train_dataset.csv?raw=True')
X_train = train_set.drop(['id'], axis=1)
X_num = X_train.drop(['Gender', 'Customer Type', 'Type of Travel', 'Class','satisfaction'], axis=1)
X_set = test_set.drop(['id'], axis=1)
y = X_train['satisfaction'].copy()

In [85]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

X_train['satisfaction'] = labelencoder.fit_transform(X_train['satisfaction'].values)
X_train['satisfaction'].value_counts()


0    5000
1    5000
Name: satisfaction, dtype: int64

In [86]:
corr_matrix = X_train.corr().abs()
corr_matrix.style.background_gradient(cmap='coolwarm')

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
Age,1.000000,0.110823,0.020904,0.036071,0.026964,0.012077,0.021971,0.215613,0.169291,0.097421,0.073412,0.060124,0.040368,0.038761,0.036921,0.062409,0.015618,0.019086,0.144302
Flight Distance,0.110823,1.000000,0.002341,0.021320,0.052807,0.017513,0.065939,0.213859,0.170203,0.150573,0.115167,0.140509,0.073161,0.066881,0.058939,0.100705,0.011425,0.002901,0.298703
Inflight wifi service,0.020904,0.002341,1.000000,0.362355,0.730731,0.362702,0.119807,0.438034,0.107020,0.198351,0.133511,0.172526,0.123695,0.039923,0.113467,0.119484,0.019843,0.023086,0.277395
Departure/Arrival time convenient,0.036071,0.021320,0.362355,1.000000,0.456498,0.479002,0.001982,0.046456,0.002102,0.007094,0.071368,0.027487,0.072969,0.078166,0.062814,0.001172,0.010846,0.008985,0.067638
Ease of Online booking,0.026964,0.052807,0.730731,0.456498,1.000000,0.476405,0.017193,0.388220,0.016831,0.043574,0.053613,0.116333,0.044259,0.013875,0.032168,0.007301,0.001474,0.003715,0.164662
Gate location,0.012077,0.017513,0.362702,0.479002,0.476405,1.000000,0.002645,0.003610,0.008359,0.001144,0.015083,0.008752,0.000493,0.037372,0.003065,0.015090,0.001907,0.000684,0.007169
Food and drink,0.021971,0.065939,0.119807,0.001982,0.017193,0.002645,1.000000,0.239007,0.573063,0.598209,0.062373,0.045929,0.030299,0.087117,0.035568,0.641701,0.011761,0.011379,0.207854
Online boarding,0.215613,0.213859,0.438034,0.046456,0.388220,0.003610,0.239007,1.000000,0.414889,0.292397,0.165847,0.133977,0.080804,0.207733,0.077808,0.341899,0.001727,0.003684,0.504986
Seat comfort,0.169291,0.170203,0.107020,0.002102,0.016831,0.008359,0.573063,0.414889,1.000000,0.615310,0.136570,0.121991,0.081107,0.192491,0.074447,0.677454,0.014833,0.013988,0.342294
Inflight entertainment,0.097421,0.150573,0.198351,0.007094,0.043574,0.001144,0.598209,0.292397,0.615310,1.000000,0.442295,0.338588,0.393506,0.109921,0.418871,0.688927,0.029500,0.029347,0.400900


In [87]:
X_train.corrwith(X_train['satisfaction']).abs().sort_values(ascending=False)

satisfaction                         1.000000
Online boarding                      0.504986
Inflight entertainment               0.400900
Seat comfort                         0.342294
On-board service                     0.326426
Leg room service                     0.324368
Cleanliness                          0.310338
Flight Distance                      0.298703
Inflight wifi service                0.277395
Baggage handling                     0.247965
Inflight service                     0.240584
Checkin service                      0.232359
Food and drink                       0.207854
Ease of Online booking               0.164662
Age                                  0.144302
Departure/Arrival time convenient    0.067638
Arrival Delay in Minutes             0.048650
Departure Delay in Minutes           0.042960
Gate location                        0.007169
dtype: float64

In [88]:
y = X_train['satisfaction'].copy()

In [89]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('std_scaler', StandardScaler())             
])

from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [90]:
X = full_pipeline.fit_transform(X_train)

In [91]:
tree_model = DecisionTreeClassifier(min_samples_leaf=5, max_depth=3)
tree_model.fit(X, y)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

In [97]:
y_pred = tree_model.predict(X)

In [98]:
from sklearn.metrics import jaccard_score

jaccard_score(y, y_pred)

0.8016601907453197

In [95]:
test_prepared = full_pipeline.fit_transform(X_set)

In [96]:
y_test = tree_model.predict(test_prepared)

satisfaction = pd.DataFrame(y_test, columns=['satisfaction'])
satisfaction.index = test_set['id']
satisfaction.to_csv('Aviakompaniya.csv')